In [1]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

import re

import stanza

In [3]:
stanza.download('ru')
nlp = stanza.Pipeline('ru')

# Основные функции

## Очистка мусора в тексте

In [4]:
def clean_text(text):
    clean_text = re.sub(r'\d+', '', text)
    clean_text = clean_text.strip()
    clean_text = re.sub(r'\b[a-zA-Zа-яА-ЯёЁ]{1}\b', '', clean_text)
    clean_text = re.sub('-\n+', '', clean_text)
    clean_text = re.sub(r'\b\d{1,2}\.\d{1,2}\.\d{4}\b', '', clean_text)
    clean_text = re.sub(r'[\xad ][\xad\n+]', '', clean_text)

    return clean_text

## Отбор кандидатов в термины
Хоть в дальнейшем и рассматривались только биграммы, но отбирать кандидатов можно и среди триграмм

In [5]:
def candidats(doc, gr_sample):
    gr_parts = gr_sample.split('+')
    gr_fr_dict = {}
    
    for sentence in doc.sentences:
        bigrams = {}

        for dependence in sentence.dependencies:
            try:
                    if dependence[0].upos == gr_parts[1] and dependence[2].upos == gr_parts[0]:
                        key = dependence[2].lemma + ' ' + dependence[0].lemma
                        if key not in bigrams:
                            bigrams[key] = 0
                        bigrams[key] += 1

            except IndexError:
                continue
 
        if len(gr_parts) == 3:
            for bigram in bigrams.keys():

                words = bigram.split(' ')
                last_word = words[1]  
             
                for dependence in sentence.dependencies:
                    if dependence[0].lemma == last_word:  
                        if dependence[2].upos == gr_parts[2]: 
                            trigram_key = words[0] + ' ' + last_word + ' ' + dependence[2].lemma
                            if trigram_key not in gr_fr_dict:
                                gr_fr_dict[trigram_key] = 0
                            gr_fr_dict[trigram_key] += 1
        elif len(gr_parts) == 2:
            for bigram in bigrams.keys():
                if bigram not in gr_fr_dict:
                    gr_fr_dict[bigram] = 0
                gr_fr_dict[bigram] += 1

    return gr_fr_dict

## Создание частотного словаря

In [6]:
def count_words(doc):
    Counter = {}
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos == 'PUNCT':
                continue
            else:
                if word.lemma not in Counter:
                    Counter[word.lemma] = 0
                Counter[word.lemma] += 1

    return Counter

## Статистические признаки

In [7]:
def MI(term, fr_term, N, fr_word_dict, alpha= 1):
    words = term.split()
    mu = 1
    for word in words:
        mu *= fr_word_dict[word]

    return np.log2(fr_term ** alpha * N / mu)

def Dice(term, fr_term, fr_word_dict):
    words = term.split()
    mu = 0
    for word in words:
        mu += fr_word_dict[word]

    return fr_term/(mu)

## Получение кандидатов в термины по разным грамматическим образцам

In [8]:
def top20by_gr_samples(doc, gr_samples, target_metric):
    res = []
    for gr_sample in gr_samples:
        cand = candidats(doc, gr_sample)
        candidats_df = pd.DataFrame(list(cand.items()), columns = ['term', 'ab_fr'])
        candidats_df['MI'] = candidats_df.apply(lambda row: MI(row['term'], row['ab_fr'], N, fr_word_dict), axis= 1)
        candidats_df['MI3'] = candidats_df.apply(lambda row: MI(row['term'], row['ab_fr'], N, fr_word_dict, alpha= 3), axis= 1)
        candidats_df['DICE'] = candidats_df.apply(lambda row: Dice(row['term'], row['ab_fr'], fr_word_dict), axis= 1)
        res.append(list(candidats_df.sort_values(by= [target_metric], ascending= False)['term'][:20]))

    return res

# Текст по механике

In [9]:
link = '/kaggle/input/russian-scientific-articles/data_3/33 Механика/elibrary_16331399_29897286.txt'
text = open(link).read()

text = clean_text(text)

In [10]:
print(len(text))
text[:1000]

29406


'\ufeffВычислительная механика сплошных сред. – . – . , № . – . -  \nУДК ..\nТЕЧЕНИЕ НЕЛИНЕЙНОЙ УПРУГОВЯЗКОЙ ЖИДКОСТИ  ПЛОСКОМКАНАЛЕ ПОД ДЕЙСТВИЕМ ЗАДАННОГО ГРАДИЕНТА ДАВЛЕНИЯ\n.. Кузнецова, .. Скульский, .. Пышнограй\n\nИнститут механики сплошных сред УрО РАН, Пермь, Россия\nАлтайский государственный технический университет, Барнаул, РоссияРассмотрено течение нелинейной вязкоупругой жидкости, характеризующейся одним тензорным\nвнутренним параметром,  плоском канале под действием постоянного перепада давления. Получены всеточные аналитические решения этой задачи  параметрическом виде, из которых выделены заведомофизически недопустимые. Найдены распределения компонент тензора анизотропии, скорости  градиентаскорости по высоте канала для различных параметров реологической модели. Показано, что для значенийперепада давления выше критических наблюдается неоднозначность решения, приводящая  разрывам  профилях компонент тензора анизотропии. Та же задача решена  двумерной постановке методомко

In [11]:
doc = nlp(text)

In [12]:
fr_word_dict = count_words(doc)
N = sum(fr_word_dict.values())
print('Количество слов в тексте: ', N)

Количество слов в тексте:  3486


In [13]:
gr_samples = ('ADJ+NOUN', 'NOUN+NOUN', 'VERB+NOUN')

In [14]:
top20by_gr_samples(doc, gr_samples, 'MI3')

[['сплошной среда',
  'вычислительный механика',
  'плоский канал',
  'упруговязкий жидкост',
  'тангенциальный разрыв',
  'технический университет',
  'экспериментальный данные',
  'поперечный координата',
  'государственный университет',
  'реологический модель',
  'нормальный составляющая',
  'продольный скорость',
  'конвективный производная',
  'макромолекулярный клубка',
  'сдвиговой вязкость',
  'материальный констант',
  'молекулярный весы',
  'относительный движениебусинка',
  'исходный параметровмодель',
  'нулевой приближение'],
 ['анизотропия тензор',
  'давление перепад',
  'среда механика',
  'тензор компонент',
  'канал высота',
  'сходимость критерий',
  'скорость профиль',
  'степень полином',
  'полимер раствор',
  'физик технич.',
  'технич. мех',
  'параметровзадача совокупность',
  'заключение привелоавтор',
  'теория приближение',
  'привелоавтор изучение',
  'оператор набла',
  'итерация известнымииз',
  'пр. сведение',
  'размер прямоугольник',
  'неоднозначныхр

In [15]:
acc1 = 17/20
acc2 = 15/20
acc3 = 7/20

print("Точность нахождения терминов:", np.mean([acc1, acc2, acc3]))

Точность нахождения терминов: 0.65


# Sapiens

In [16]:
link = '/kaggle/input/sapiens/Harari_Sapiens-Kratkaya-istoriya-chelovechestva.OGmOcQ.504097.txt'
text = open(link).read()

text = clean_text(text)

In [17]:
print(len(text))
text[:1000]

817730


'Annotation\n\n\nСто тысяч лет назад Homo sapiens был одним из как минимум шести видов человека, живших на этой планете, – ничем не примечательным животным, которое играло  экосистеме роль не большую, чем гориллы, светлячки или медузы. Но около семидесяти тысяч лет назад загадочное изменение когнитивных способностей Homo sapiens превратило его  хозяина планеты  кошмар экосистемы. Как человек разумный сумел покорить мир? Что стало  другими видами человека? Когда  почему появились деньги, государства  религия? Как возникали  рушились империи? Почему почти все общества ставили женщин ниже мужчин? Как наука  капитализм стали господствующими вероучениями современной эры? Становились ли люди  течением времени счастливее? Какое будущее нас ожидает?\n\nЮваль Харари показывает, как ход истории формировал человеческое общество  действительность вокруг него. Его книга прослеживает связь между событиями прошлого  проблемами современности  заставляет читателя пересмотреть все устоявшиеся представле

In [18]:
doc = nlp(text)

In [19]:
fr_word_dict = count_words(doc)
N = sum(fr_word_dict.values())
print('Количество слов в тексте: ', N)

Количество слов в тексте:  102139


In [20]:
gr_samples = ('ADJ+NOUN', 'NOUN+NOUN', 'VERB+NOUN', 'ADV+ADJ')

In [21]:
top20by_gr_samples(doc, gr_samples, 'MI3')

[['сельский хозяйство',
  'аграрный революция',
  'генный инженерия',
  'железный дорога',
  'xix век',
  'живой существо',
  'порочный круг',
  'квантовый механика',
  'самый дело',
  'современный эпоха',
  'квадратный километр',
  'детский смертность',
  'когнитивный революция',
  'атомный бомба',
  'естественный отбор',
  'сахарный тростник',
  'тысячелетний давность',
  'тихий океан',
  'промышленный революция',
  'воображаемый порядок'],
 ['зрение точка',
  'собиратель охотник',
  'друг друг',
  'тысяча год',
  'авт прим.',
  'чиля перец',
  'молчание завес',
  'серебро сикель',
  'пенс шиллинг',
  'ткач землевладелец',
  'мобиль перпетуум',
  'чулок парик',
  'каблук парик',
  'шпион муха',
  'каури раковина',
  'община семья',
  'обстоятельство стечение',
  'закат рассвет',
  'приключение искатель',
  'грант заявка'],
 ['окружать среда',
  'просигнать гудка',
  'вживить макака',
  'абортировать зародыша',
  'обуглить тушка',
  'возносить похвала',
  'выдубить патриарх',
  'капри

In [22]:
acc1 = 16/20
acc2 = 14/20
acc3 = 3/20
acc4 = 9/20

print("Точность нахождения терминов:", np.mean([acc1, acc2, acc3, acc4]))

Точность нахождения терминов: 0.525
